<a href="https://colab.research.google.com/github/lindaelmi/Tasks/blob/main/task2_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка зависимостей
!pip install -q nltk pymorphy2 razdel scikit-learn gensim

# Импорт библиотек
import nltk, os, random, numpy as np
import razdel                         # сегментация русского текста
from nltk.corpus import stopwords
!pip install -q pymorphy3
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from gensim.downloader import load

nltk.download('stopwords')
stop_ru = set(stopwords.words('russian'))
morph   = MorphAnalyzer()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 48.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
%%writefile corpus.txt
привет, мир! я люблю машинное обучение.
здравствуйте, друзья! искусственный интеллект – будущее.
привет! нейросети крутые.
здравствуйте! я изучаю глубокое обучение.
приветствую! компьютерное зрение – моя страсть.
здравствуйте! анализ текста очень интересен.
привет! NLP – лучшая область.
здравствуйте! я читаю статьи о ML.
привет! transformers – топ.
здравствуйте! я пишу код на питоне.

Writing corpus.txt


In [ ]:
def preprocess(text):
    """Лемматизация + удаление стоп-слов"""
    tokens = [t.text for t in razdel.tokenize(text)]
    lemmas = [morph.parse(t)[0].normal_form for t in tokens]
    return " ".join([l for l in lemmas if l not in stop_ru])

# читаем corpus
with open("corpus.txt", encoding="utf-8") as f:
    lines = f.readlines()

# labels artificiels : 0 = привет, 1 = tech
labels = [0 if "привет" in l.lower() else 1 for l in lines]

# pré-traitement
texts = [preprocess(t) for t in lines]
print("Пример после обработки:", texts[0])

Пример после обработки: привет , мир ! любить машинный обучение .


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)
print(f"Train: {len(X_train)} | Test: {len(X_test)}")

Train: 8 | Test: 2


In [ ]:
# токенизация для Word2Vec
sentences = [s.split() for s in X_train]

w2v_model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4, epochs=10)
print("Word2Vec обучен ✅")

Word2Vec обучен ✅


In [ ]:
def vectorize_avg(texts, model):
    """Среднее всех векторов слов документа"""
    vectors = []
    for sent in texts:
        word_vecs = [model.wv[w] for w in sent if w in model.wv]
        if word_vecs:
            vectors.append(np.mean(word_vecs, axis=0))
        else:
            vectors.append(np.zeros(model.vector_size))
    return np.array(vectors)

X_train_avg = vectorize_avg(X_train, w2v_model)
X_test_avg  = vectorize_avg(X_test,  w2v_model)

In [ ]:
clf = LinearSVC()
clf.fit(X_train_avg, y_train)
y_pred = clf.predict(X_test_avg)

print("🔹 Moyenne simple – Word2Vec")
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["привет", "tech"]))

🔹 Moyenne simple – Word2Vec
Accuracy: 0.5
              precision    recall  f1-score   support

      привет       0.50      1.00      0.67         1
        tech       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# TF-IDF sur le même corpus
tfidf = TfidfVectorizer(lowercase=False, tokenizer=lambda x: x.split())
tfidf.fit(X_train)

def vectorize_tfidf(texts, w2v_model, tfidf_model):
    vectors = []
    for sent in texts:
        words = sent.split()
        weights = tfidf_model.transform([" ".join(words)]).toarray()[0]
        indices = tfidf_model.vocabulary_
        weighted_vecs = []
        for w in words:
            if w in indices and w in w2v_model.wv:
                idx = indices[w]
                weighted_vecs.append(weights[idx] * w2v_model.wv[w])
        if weighted_vecs:
            vectors.append(np.mean(weighted_vecs, axis=0))
        else:
            vectors.append(np.zeros(w2v_model.vector_size))
    return np.array(vectors)

X_train_tfidf = vectorize_tfidf(X_train, w2v_model, tfidf)
X_test_tfidf  = vectorize_tfidf(X_test,  w2v_model, tfidf)

clf_tfidf = LinearSVC()
clf_tfidf.fit(X_train_tfidf, y_train)
y_pred_tfidf = clf_tfidf.predict(X_test_tfidf)

print("\n🔹 Moyenne TF-IDF – Word2Vec")
print("Accuracy:", accuracy_score(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_tfidf, target_names=["привет", "tech"]))


🔹 Moyenne TF-IDF – Word2Vec
Accuracy: 1.0
              precision    recall  f1-score   support

      привет       1.00      1.00      1.00         1
        tech       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
print("Сравнение:")
print(f"Среднее simple : {accuracy_score(y_test, y_pred):.2f}")
print(f"Среднее TF-IDF : {accuracy_score(y_test, y_pred_tfidf):.2f}")

Сравнение:
Среднее simple : 0.50
Среднее TF-IDF : 1.00
